In [443]:
import requests
from sklearn.manifold import TSNE
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz, process


In [586]:
#loading block
mykey = open('spoonac/apikey.txt').read().strip()
w2vm = pickle.load(open("bigoven/w2vmodel.pkl", 'rb'))
aisledict = pickle.load(open("bigoven/aisleclassification.pkl", 'rb'))
noise = pickle.load(open("bigoven/noiselist.pkl", 'rb'))

atFM = pickle.load(open("bigoven/FMproducts.pkl", 'rb'))
FMinfo = pd.read_excel('greenmarketproducts.xlsx')
FMinfo["aisles"] = FMinfo["aisles"].str.lower()
FMinfo['TYPES OF PRODUCTS AVAILABLE'] = FMinfo['TYPES OF PRODUCTS AVAILABLE'].str.lower()
veggies = open('juneveggies.txt').read().strip()

In [604]:
okay = FMinfo.copy()
okay['TYPES OF PRODUCTS AVAILABLE'] = veggies 
FMinfo = FMinfo.where(FMinfo['TYPES OF PRODUCTS AVAILABLE'] != 'vegetables', okay)
#pickle.dump(FMinfo,open("bigoven/FMfull.pkl","wb"))




In [624]:
#from the user
#userinput = 'https://www.simplyrecipes.com/recipes/chicken_piccata/'
userinput = 'https://tasty.co/recipe/chicken-veggie-stir-fry' # 1
#userinput = 'https://www.simplyrecipes.com/recipes/lasagna/'
#userinput = 'https://www.simplyrecipes.com/recipes/potato_leek_soup/'
#userinput = 'https://damndelicious.net/2019/08/06/easy-chicken-tacos/' # 2
#userinput = 'https://www.thewholesomedish.com/the-best-classic-burger/'
params = {'url': userinput, 'forceExtraction': 'true', 'apiKey': mykey, 'analyze': 'true'}

In [625]:
#get infor about recipe they requested
response = requests.get('https://api.spoonacular.com/recipes/extract', params=params)


In [626]:
#extract ingredients from their recipe
rec = response.json()
ingcomp = rec['extendedIngredients']
ingredients = [','.join([ing['name'].lower() for ing in ingcomp if ing['name'] ])] #add the stripping



#print(repr(ingredients)) # repr is the representation 
ingredients

['chicken breast,salt,bell pepper,broccoli florets,mushroom,oil,garlic,ginger,sesame oil,reduced sodium soy sauce,brown sugar,chicken broth,flour']

In [627]:
#suite of deduping functions

#on loading we tokenize, strip punctuation&digits&spaces and make lowercase 

def removenoise(input_ing):
    noise_free_ing = []
    print(input_ing)
    for word in input_ing.split(','):
        checked = []
        splitit = word.split()
        checked.extend(i for i in splitit if i not in noise)
        noise_free_ing.append(' '.join(checked))
    return noise_free_ing
        


In [628]:
#call the dedupe functions
# nning = []
# for i in ingredients:
#     nning.append(removenoise(i))
nning = removenoise(ingredients[0])

    


chicken breast,salt,bell pepper,broccoli florets,mushroom,oil,garlic,ginger,sesame oil,reduced sodium soy sauce,brown sugar,chicken broth,flour


In [629]:

nning

['chicken breast',
 'salt',
 'bell pepper',
 'broccoli florets',
 'mushroom',
 'oil',
 'garlic',
 'ginger',
 'sesame oil',
 'reduced sodium soy sauce',
 'brown sugar',
 'chicken broth',
 'flour']

In [633]:
#go through ingredients, if theyre not available at the FM use w2v and aisle to find similar 
#available = []
#toreplace = []
#replacement = []

output = {
    'ingredient': None,
    'where_available': None,
    'unknown': None,
    'baking': False, #true if is baking
    'spices': False, #true if spice/seasoning
    'spice_businesses': None,
    'match': None,
    'try_fresh': None,
    'store_match': None
}

allout = []

for i in nning:
    thisout = output.copy()
    thisout['ingredient'] = i
    
    highest = process.extractOne(i,atFM)
    if highest[1] >= 90:
        matchaisle = FMinfo.loc[FMinfo['TYPES OF PRODUCTS AVAILABLE'].str.contains(highest[0])]
        thebiz = matchaisle['BUSINESS NAME'].tolist()
        thisout['where_available'] = thebiz
        print(f'{i} is available at {thebiz} ')
        
    else:
        #these are unavailable ingredients
        curaisle = aisledict.get(i)
        
        #use w2v to find similar ingredients
        try:
            similar = w2vm.wv.most_similar(i, topn=100)
            opposite = w2vm.wv.most_similar(similar[0][0], topn=1000)
            toreplace.append(i)
            print(f'{i} is not but ')
        except KeyError:
            thisout['unknown'] = i
            print(f'We have never heard of {i}, sorry about that')
            continue
            
        #get rid of baking for now
        if curaisle[0] is not None and curaisle[0] == 'Baking':
            thisout['baking'] = True
            print(f'{i} is a baking product, which is likely in your pantry!')
            continue
            
        #deal with the seasoning issue
        if curaisle[0] is not None and curaisle[0] == 'Spices and Seasonings':
            matchaisle = FMinfo.loc[FMinfo['aisles'] == curaisle[0].lower()]
            thebiz = matchaisle['BUSINESS NAME'].tolist()
            #print(f'Dried Spices and and seasonings are rare, you may have this in your pantry, otherwise get fresh ones at: {thebiz}\n')
            thisout['spices'] = True
            thisout['spice_businesses'] = thebiz
            continue
            
        # here is thing our algorithm thinks is similar and IS available
        item = []
        for opp in opposite:
            ophighest = process.extractOne(opp[0],atFM)
            if ophighest[1] >= 90:
                item.append(opp[0])
                if len(item) == 1:
                    break
        print(f'Here is the item our algorithm thinks is most similar and is available: {item}\n')
        thisout['match'] = item
        
        # if it is something usually prepackaged, suggest making it fresh
        if curaisle[0] == 'Pasta and Rice' or curaisle[0] == 'Canned and Jarred':
            for sim in similar:
                a = aisledict[sim[0]]
                if a[0] == 'Produce':
                    trythis = sim[0]
                break
            print(f'Canned/Jarred items are rare at the Market, but you can make this fresh using {trythis}\n')
            thisout['try_fresh'] = trythis
                
        if curaisle[0] is not None: 
            #if something is not available, this vendor might be able to help you
            matchaisle = FMinfo.loc[FMinfo['aisles'] == curaisle[0].lower()]
            thebiz = matchaisle['BUSINESS NAME'].tolist()
            print(f'This list of vendors often has products similar to {i}, try asking them: {thebiz}\n')
            thisout['store_match'] = thebiz



    allout.append(thisout)
        
        
        

            
            
#print(allout)



chicken breast is available at ['Abundance Acres Farm', 'Feisty Acres Inc', 'flying pig farms', 'green table farms', 'Hawthorne Valley Farm', 'Norwich Meadows Farm', "Quattro's Game Farm", 'Ramble Creek Farm', 'Sawkill Farm', 'Shannon Brook Farm', 'Stony Mountain Ranch', 'Sun Fed Beef and Pork', 'Violet Hill Farm', 'Yellow Bell Farm'] 
salt is not but 
bell pepper is not but 
Here is the item our algorithm thinks is most similar and is available: ['zucchini']

This list of vendors often has products similar to bell pepper, try asking them: ['Alewife Farm', 'Blue Oyster Cultivation', 'breezy hill orchard', 'Brooklyn Grange Rooftop Farm', 'Bulich Mushroom Co.', 'Campo Rosso Farm', 'caradonna farms', 'Cherry Lane Farms', "D'Attolico's Organic", 'Down Home Acres', 'Eckerton Hill Farm', 'Element Farms Inc', 'Evolutionary Organics', 'Feisty Acres Inc', 'Fishkill Farms', 'Fledging Crow Vegetables', 'Garden of Eve Organic Farm', 'gopal farms', 'Gorzynski Ornery Farm', 'Heritage Farm at Snug Ha

In [601]:
aisledict['reduced sodium soy sauce']

['Ethnic Foods;Condiments']

In [351]:
curing = 'lasagna noodles'
curaisle = aisledict[curing]
similar = w2vm.wv.most_similar(curing, topn=10)
opposite =  w2vm.wv.most_similar(similar[0][0], topn=10)
print(opposite)
# thebiz = FMinfo.loc[FMinfo['aisles'] == curaisle[0].lower()]
# bestbiz = []
# for biz in thebiz['TYPES OF PRODUCTS AVAILABLE']:
#     bizlist = biz.split(',')
#     for ing in bizlist:
#         w2vsim = w2vm.wv.similarity(opp)
    
#     #sm = model.similarity('france', 'spain')
#     #similar = w2vm.wv.most_similar(i, topn=1)

#     print(inbiz)


# a


[('lasagna noodles', 0.9908419251441956), ('mozzarella cheese', 0.9805896282196045), ('mozzarella', 0.979179859161377), ('ziti', 0.9665677547454834), ('romano cheese', 0.9635792970657349), ('seasoning', 0.9611002802848816), ('manicotti', 0.9600947499275208), ('lasagne noodles', 0.9597089886665344), ('marinara sauce', 0.9596649408340454), ('milk mozzarella cheese', 0.9572638273239136)]


In [278]:
biz.split(',')
#w2vm.wv.similarity(curing, 'tomato sauces')

['Vegetables',
 ' Fruit',
 ' Beef',
 ' Pork',
 ' Poultry',
 ' Lamb',
 ' Salsa',
 ' pasta sauce',
 ' Marinara ']

In [76]:
fuzz.ratio(curing, opposite[0][0])

21

In [280]:
'cheese' in atFM

True

In [513]:
aaa = w2vm.wv.most_similar('bell pepper', topn=1)
print(aaa)
w2vm.wv.most_similar(aaa[0][0], topn=100)

#w2vm.wv.similarity

[('bell peppers', 0.9264739751815796)]


[('bell pepper', 0.92647385597229),
 ('portobello mushroom cap', 0.9045850038528442),
 ('squash', 0.8921389579772949),
 ('zucchini', 0.8898555040359497),
 ('vegetable broth', 0.8821101784706116),
 ('grassfed beef', 0.8798379302024841),
 ('tomato', 0.8782333135604858),
 ('tomatoes', 0.8773675560951233),
 ('quinoa', 0.8754387497901917),
 ('brown lentils', 0.8680348992347717),
 ('mince', 0.8655365705490112),
 ('petitediced tomatoes', 0.8645636439323425),
 ('wheat couscous', 0.8637833595275879),
 ('butternut', 0.8633012771606445),
 ('beef extra', 0.8593685030937195),
 ('onions wedges', 0.855331301689148),
 ('ounces tomato paste', 0.8549329042434692),
 ('chick peas', 0.8546891808509827),
 ('orange bell pepper', 0.8528242111206055),
 ('okra', 0.8527382612228394),
 ('pinch saffron', 0.8459432125091553),
 ('steak mince', 0.8440986275672913),
 ('cherry pepper', 0.8436112999916077),
 ('roma tomatoes', 0.8423422574996948),
 ('yam', 0.8409353494644165),
 ('peppers', 0.8403393030166626),
 ('cous co

In [48]:
fuzz.ratio('broccoli florets', 'broccoli')

67

In [352]:
aisledict['sugar']

['Baking']

In [220]:
FMinfo['aisles']

0               Alcoholic Beverages
1                              meat
2                           produce
3      Nut butters, jams, and honey
4               Alcoholic Beverages
                   ...             
133                         produce
134    nut butters, Jams, and Honey
135                         produce
136                            meat
137               canned and jarred
Name: aisles, Length: 138, dtype: object

In [258]:
#FMinfo['aisles'].loc
a

,BUSINESS NAME,TYPES OF PRODUCTS AVAILABLE,aisles
26,City Saucery,"Tomato Sauces, Canned Tomatoes, Pickles, Pasta",pasta and rice
49,GrowNYC Grainstand,"grains, flour, beans, pasta",pasta and rice
63,Knoll Krest Farm,"Eggs, Pasta",pasta and rice
85,Ox Hollow Farm,"Vegetables, Fruit, Beef, Pork, Poultry, Lamb, ...",pasta and rice


In [257]:
curaisle[0]

In [353]:
curaisle = 'Baking'

In [354]:
matchaisle = FMinfo.loc[FMinfo['aisles'] == curaisle.lower()]
matchaisle


,BUSINESS NAME,TYPES OF PRODUCTS AVAILABLE,aisles


In [347]:
curaisle.lower()

'spices and seasonings'

In [370]:
matchaisle = FMinfo.loc[FMinfo['TYPES OF PRODUCTS AVAILABLE'].split(',') == 'beef']
matchaisle

AttributeError: 'Series' object has no attribute 'split'

In [385]:
FMinfo.loc[FMinfo['TYPES OF PRODUCTS AVAILABLE'].strcontains('beef')]

AttributeError: 'Series' object has no attribute 'strcontains'

In [450]:
highest = 'tomato sauces'
matchaisle = FMinfo.loc[FMinfo['TYPES OF PRODUCTS AVAILABLE'].str.contains(highest)]


In [467]:
aisledict('seasoning')

TypeError: 'dict' object is not callable

In [390]:
w2vm.wv.most_similar('extra virgin olive oil')

'olive oil' in atFM

[('olive oil', 0.9594991207122803),
 ('pine nuts', 0.9251871109008789),
 ('kalamata olives', 0.9077877998352051),
 ('caper', 0.901020884513855),
 ('plum tomatoes', 0.9004257917404175),
 ('extravirgin olive oil', 0.8953660726547241),
 ('parsley cilantro', 0.8888424634933472),
 ('feta cheese', 0.8852983713150024),
 ('mild turkey sausage', 0.8838472366333008),
 ('cherry tomatoes', 0.8764583468437195)]

In [395]:
process.extractOne('extra virgin olive oil',atFM)

('gin', 90)

In [420]:
FMinfo['TYPES OF PRODUCTS AVAILABLE'].str.lower()

AttributeError: 'DataFrame' object has no attribute 'str'